In [45]:
from bs4 import BeautifulSoup;
import urllib2;
import re;
import requests;

# Get all of the Urls

In [46]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt;
import grequests;

In [47]:
data = pd.DataFrame.from_csv("C:/R/990 form/CLEAN_DATA/Sample_Program_Service_Data.csv")

In [48]:
data.columns

Index([u'EIN', u'GROSSRECEIPTS', u'TOTEMPLOYEE', u'TOTVOLUNTEERS', u'CITY',
       u'STATE', u'FISYR', u'TOTALPROGSERVEXP', u'FORMTYPE', u'FORMYEAR',
       u'NAME', u'DBA', u'ADDRESS', u'ZIP', u'WEBSITE', u'URL', u'NteeFinal',
       u'MISSION', u'DISCOPS', u'CONTRIBCURRENT', u'PSRCURRENT',
       u'INVINCCURRENT', u'OTHERREVCURRENT', u'TOTALREVCURRENT', u'MEMBERDUES',
       u'GROSSSALESOTHER', u'SALESCOSTOTHER', u'NETSALESOTHER',
       u'GROSSINCGAMING', u'GROSSINCFNDEVENTS', u'NETSALESINV',
       u'GRANTSPAIDCURRENT', u'MEMBERBENCURRENT', u'SALARIESCURRENT',
       u'TOTALEXPCURRENT', u'REVLESSEXPCURRENT', u'TOTALASSETSBEGYEAR',
       u'TOTALASSETSENDYEAR', u'TOTALLIABBEGYEAR', u'TOTALLIABENDYEAR',
       u'NETASSETSBEGYEAR', u'OTHERASSETSCHANGES', u'NETASSETSENDYEAR',
       u'CASHINVBEGYEAR', u'CASHINVENDYEAR', u'LANDBEGYEAR', u'LANDENDYEAR',
       u'OTHERASSETSBEGYEAR', u'OTHERASSETSENDYEAR', u'LOBBYING', u'proj1Desc',
       u'proj1ExpAmt', u'proj1GrantAmt', u'proj2Desc', u

In [49]:
data.head(5)[["WEBSITE", "NAME", "ADDRESS", "ZIP", "URL", "FISYR"]]

,WEBSITE,NAME,ADDRESS,ZIP,URL,FISYR
1,WWW.ROCHESTERBIZ.COM,GREATER ROCHESTER ENTERPRISE,100 CHESTNUT ONE HSBC PLAZA NO 1910,14604,https://s3.amazonaws.com/irs-form-990/20161124...,2015
2,WWW.LJBI.ORG,LA JOLLA BIOENGINEERING INSTITUTE,505 COAST BOULEVARD SOUTH,92037,https://s3.amazonaws.com/irs-form-990/20153349...,2014
3,NaN,ACTS COMMUNITY DEVELOPMENT,1034 66TH AVENUE,94621,https://s3.amazonaws.com/irs-form-990/20151313...,2014
4,NaN,AMERICAN AMISTAD FOUNDATION INC,733 SHINAVA DRIVE,84738,https://s3.amazonaws.com/irs-form-990/20151288...,2014
5,WWW.CASTLA.ORG,COALITION TO ABOLISH SLAVERY AND TRAFFICKING,5042 WILSHIRE BLVD NO 586,90036,https://s3.amazonaws.com/irs-form-990/20160137...,2014


In [50]:
float(len(data[pd.isnull(data.WEBSITE)])) / len(data)

0.2961174153615305

In [51]:
OkayWebsites = data[pd.isnull(data.WEBSITE) == False]
OkayWebsites.head(5)[["WEBSITE", "NAME", "ADDRESS", "ZIP", "URL", "FISYR"]]

,WEBSITE,NAME,ADDRESS,ZIP,URL,FISYR
1,WWW.ROCHESTERBIZ.COM,GREATER ROCHESTER ENTERPRISE,100 CHESTNUT ONE HSBC PLAZA NO 1910,14604,https://s3.amazonaws.com/irs-form-990/20161124...,2015
2,WWW.LJBI.ORG,LA JOLLA BIOENGINEERING INSTITUTE,505 COAST BOULEVARD SOUTH,92037,https://s3.amazonaws.com/irs-form-990/20153349...,2014
5,WWW.CASTLA.ORG,COALITION TO ABOLISH SLAVERY AND TRAFFICKING,5042 WILSHIRE BLVD NO 586,90036,https://s3.amazonaws.com/irs-form-990/20160137...,2014
6,HTTP://WALKRUNFORLIFE.COM/,ST LAWRENCE COUNTY CANCER FUND INC,PO BOX 291,13617,https://s3.amazonaws.com/irs-form-990/20164132...,2015
7,WWW.BRIDGTONHOSPITAL.ORG,BRIDGTON HOSPITAL,29 LOWELL STREET BOX A,4240,https://s3.amazonaws.com/irs-form-990/20160133...,2014


In [52]:
def exception_handler(request, exception):
    None
    #print "Request failed: " + str(request.url) + "; Exception: " + str(exception)

In [53]:
def ProcessText(text):
    soup = BeautifulSoup(text);
    return map(lambda link: link.get('href'), soup.find_all('a', attrs={'href': re.compile('twitter.com')}));

def ProcessRequest(requestsPerWebsite):
    #print(requestsPerWebsite);
    if (requestsPerWebsite[0] is not None and requestsPerWebsite[0].ok):
        return ProcessText(requestsPerWebsite[0].text);
    if (len(requestsPerWebsite) == 2):
        if (requestsPerWebsite[1] is not None and requestsPerWebsite[1].ok):
            return ProcessText(requestsPerWebsite[1].text);
    return [];

In [54]:
hdr = {'User-Agent': 'Mozilla/5.0'}
ishttp = re.compile('^http(s*):\/\/.*', flags=re.IGNORECASE)
def QueryWebsite(website):
    if (ishttp.match(website) is None):
        requests = ["http://" + website, "https://" + website];
    else:
        requests = [ website ];
    return list(grequests.get(u, timeout=4) for u in requests)

def ProcessWebsiteBatch(websiteBatch):
    unsentRequests = list(map(lambda x: QueryWebsite(x), websiteBatch));
    unsentRequestLengths = list(map(lambda x: len(x), unsentRequests));
    flattenedRequests = []
    for unsent in unsentRequests:
        flattenedRequests = flattenedRequests + unsent;
    
    print "Flattened Requests: " + str(len(flattenedRequests))
    
    reqs = grequests.map(flattenedRequests, exception_handler=exception_handler)
    idx = 0;
    unFlattenedResponses = []
    for l in unsentRequestLengths:
        unFlattenedResponses = unFlattenedResponses + [reqs[idx : idx + l]];
        idx = idx + l;

    # print "Unflattened Requests: " + str(len(unFlattenedResponses))
    
    return map(lambda x: ProcessRequest(x), unFlattenedResponses);

In [55]:
def BatchColumnRun(column, batchSize):    
    goodWebsites = column;
    finalResult = [];
    finalNum = (len(goodWebsites) / batchSize);
    if (len(goodWebsites) % batchSize <> 0):
        finalNum = finalNum + 1
    for i in range(0, finalNum):
        division = goodWebsites[i*batchSize:min((i+1)*batchSize, len(goodWebsites))];
        twitterResults = ProcessWebsiteBatch(division);
        finalResult = finalResult + twitterResults
        print "Done: " + str(i)
    return finalResult;

In [ ]:
OkayWebsites['TwitterHandles'] = BatchColumnRun(OkayWebsites['WEBSITE'], 25)

Flattened Requests: 47
Done: 0
Flattened Requests: 50
Done: 1
Flattened Requests: 49
Done: 2
Flattened Requests: 48
Done: 3
Flattened Requests: 49
Done: 4
Flattened Requests: 48
Done: 5
Flattened Requests: 48
Done: 6
Flattened Requests: 49
Done: 7
Flattened Requests: 49


In [ ]:
OkayWebsites[["EIN", "TwitterHandles", "FISYR"]].to_csv("TwitterHandleData.csv")